In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.4.1


In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 246908465493731548, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14674281152
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15645767896973485500
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
from collections import Counter
import itertools
import shutil
import random
import glob
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)
%matplotlib inline

In [5]:
!pip install python-chess

In [6]:
import chess
import chess.pgn

In [7]:
from google.colab import drive
drive.mount('/content/Pdrive')

Drive already mounted at /content/Pdrive; to attempt to forcibly remount, call drive.mount("/content/Pdrive", force_remount=True).


In [8]:
find_num = {'a1': chess.A1, 'a2': chess.A2, 'a3': chess.A3, 'a4': chess.A4, 'a5': chess.A5, 'a6': chess.A6, 'a7': chess.A7,
            'a8': chess.A8, 'b1': chess.B1, 'b2': chess.B2, 'b3': chess.B3, 'b4': chess.B4, 'b5': chess.B5, 'b6': chess.B6,
            'b7': chess.B7, 'b8': chess.B8, 'c1': chess.C1, 'c2': chess.C2, 'c3': chess.C3, 'c4': chess.C4, 'c5': chess.C5,
            'c6': chess.C6, 'c7': chess.C7, 'c8': chess.C8, 'd1': chess.D1, 'd2': chess.D2, 'd3': chess.D3, 'd4': chess.D4, 
            'd5': chess.D5, 'd6': chess.D6, 'd7': chess.D7, 'd8': chess.D8, 'e1': chess.E1, 'e2': chess.E2, 'e3': chess.E3,
            'e4': chess.E4, 'e5': chess.E5, 'e6': chess.E6, 'e7': chess.E7, 'e8': chess.E8, 'f1': chess.F1, 'f2': chess.F2,
            'f3': chess.F3, 'f4': chess.F4, 'f5': chess.F5, 'f6': chess.F6, 'f7': chess.F7, 'f8': chess.F8, 'g1': chess.G1,
            'g2': chess.G2, 'g3': chess.G3, 'g4': chess.G4, 'g5': chess.G5, 'g6': chess.G6, 'g7': chess.G7, 'g8': chess.G8,
            'h1': chess.H1, 'h2': chess.H2, 'h3': chess.H3, 'h4': chess.H4, 'h5': chess.H5, 'h6': chess.H6, 'h7': chess.H7,
            'h8': chess.H8}
        

In [9]:
def data_augment(list_in, str_from): 
    
    # augments the board position when it's black's turn to make it appear as if it's white's turn.
    # for this, we just need to rotate the board by 180 degrees anti-clock and flip the color of all the pieces.
    
    list_out = ['null', 'null', 'null', 'null', 'null', 'null', 'null', 'null']
    
    var = 7
    
    if len(list_in) == 8:
        
        for el in list_in:
            el_rev = el[::-1]
            fin = ""
            for char in el_rev:
                if char.isalpha():
                    if char.islower():
                        fin += char.upper()
                    elif char.isupper():
                        fin += char.lower()
                elif char.isdigit():
                    fin += char
            
            list_out[var] = fin
            var = var - 1
              
    else:
        print('Error: List not compatible')
    
    fin1 = ""
    
    alpha = list(str_from)
    fin1 += chr(8 - ord(alpha[0]) + ord('a') -1 + 97)
    fin1 += str(9 - int(alpha[1]))
    num = find_num[fin1]
    
    return list_out, num

In [10]:
def count_total_moves(game):
    
    # counts total number of moves of any game.
    
    count = 0
    for posn in game.mainline():
        count += 1 
    
    return count

In [11]:
def fen_to_matrix(string_fen, string_move):
    
    # for a given board in FEN notation, it converts that to a corresponding 8x8x6 matrix board along with it's output label.
    # The output label can be the position from where move was made or the position to where the move was made. It'll be a
    # number between 0 to 63 corresponding to the given string- string_move.
    
    board_matrix = np.zeros((8, 8, 6))
    
    fen_list = string_fen.split(" ")[0].split("/")
    val_ = 0
    
    for x in fen_list:
        temp_s = ""
        for y in x:
            if y.isalpha():
                temp_s += y
            elif y.isdigit():
                for al_ in range(int(y)):
                    temp_s += y
                    
        fen_list[val_] = temp_s
        val_ += 1  
        
    next_ = string_fen.split(" ")[1]
    
    
    
    if next_ == "w":
        label = find_num[string_move]
        
        for i in range(len(fen_list)):
            for j in range(len(fen_list[i])):
                if fen_list[i][j].isalpha():
                    board_matrix[i, j, notation[fen_list[i][j]][0]] = notation[fen_list[i][j]][1]    
    elif next_ == "b":
        fen_list, label = data_augment(fen_list, string_move)
        
        for i in range(len(fen_list)):
            for j in range(len(fen_list[i])):
                if fen_list[i][j].isalpha():
                    board_matrix[i, j, notation[fen_list[i][j]][0]] = notation[fen_list[i][j]][1]
                    
    
    return board_matrix, label

In [12]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from collections import Counter

In [13]:
def encoding_label(y_in):
    
    # encodes the output vector (from 0 to 63 in our case) 
    
    encoder = LabelEncoder()
    encoder.fit(y_in)
    encoded_y = encoder.transform(y_in)
    
    return encoded_y

In [14]:
def tt_split(Xin, yin):
    
    # does train-test split using stratified KFold cross validation method.
    
    skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 1)    # 80% - 20% split.
    
    skf.get_n_splits(Xin, yin)
    
    for train_index, test_index in skf.split(Xin, yin):
        X_train, X_test = Xin[train_index], Xin[test_index]
        y_train, y_test = yin[train_index], yin[test_index]
        
    return X_train, X_test, y_train, y_test

In [15]:
path = '/content/Pdrive/MyDrive/Colab Notebooks/FICS_Database.pgn'
file_op = open(path)

In [16]:
list_games = []

while True:
    readd = chess.pgn.read_game(file_op)
    if readd is None:
        break
    
    list_games.append(readd)

In [17]:
print(len(list_games))  # total 5794 games have been taken as the dataset. Each game would contain 60-160 moves on an avg.

5794


In [18]:
notation = {'p': (0, -1),
            'n': (1, -1),
            'b': (2, -1),
            'r': (3, -1),
            'q': (4, -1),
            'k': (5, -1),
            'P': (0, 1),
            'N': (1, 1),
            'B': (2, 1),
            'R': (3, 1),
            'Q': (4, 1),
            'K': (5, 1)
             }

In [19]:
# preparing dataset for the 'from CNN'

boards_list = []
moves_list = []

for gm in list_games:
    reg1 = 0
    reg2 = 0
    countt = count_total_moves(gm)
    moves_list_temp = []
    
    for pos in gm.mainline():
        reg1 += 1
        if reg1!=1:
            moves_list_temp.append(str(pos.move)[0] + str(pos.move)[1]) # the position was appended from where the move was made
            
    for posi in gm.mainline():
        reg2 += 1
        if reg2!=countt:
            gamma0, gamma1 = fen_to_matrix(posi.board().fen(), moves_list_temp[reg2-1])
            boards_list.append(gamma0)
            moves_list.append(gamma1)
                

In [20]:
X_fromCNN = np.array(boards_list)

In [21]:
y_fromCNN = np.array(moves_list)

In [25]:
print(X_fromCNN.shape)
print(y_fromCNN.shape)

(806539, 8, 8, 6)
(806539,)


In [23]:
# Encoding the labels (or the possible outputs) using the above-made function.

y_test_from = encoding_label(y_fromCNN)

In [24]:
# Train-Test split:

X_train_from, X_test_from, y_train_from, y_test_from = tt_split(np.array(boards_list), y_test_from)

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.regularizers import l1, l2, l1_l2

In [30]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (2, 2), activation = 'elu', kernel_regularizer = l2(0.000005), input_shape = (8, 8, 6)))
model.add(BatchNormalization())
model.add(Dropout(0.15))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'elu', kernel_regularizer = l2(0.000005)))
model.add(BatchNormalization())
model.add(Dropout(0.15))
model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'elu', kernel_regularizer = l2(0.000005)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(units = 32, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dense(units = 64, activation = 'softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 7, 7, 32)          800       
_________________________________________________________________
batch_normalization (BatchNo (None, 7, 7, 32)          128       
_________________________________________________________________
dropout (Dropout)            (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 64)          18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 5, 64)          256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 128)        

In [33]:
model.compile(optimizer = Adam(learning_rate = 0.0001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(x = X_train_from, y = y_train_from, batch_size = 128, epochs = 20)

Epoch 1/20
5041/5041 [==============================] - 22s 4ms/step - loss: 1.5258 - accuracy: 0.4536
Epoch 2/20
5041/5041 [==============================] - 22s 4ms/step - loss: 1.5217 - accuracy: 0.4545
Epoch 3/20
5041/5041 [==============================] - 22s 4ms/step - loss: 1.5176 - accuracy: 0.4550
Epoch 4/20
5041/5041 [==============================] - 21s 4ms/step - loss: 1.5156 - accuracy: 0.4536
Epoch 5/20
5041/5041 [==============================] - 21s 4ms/step - loss: 1.5198 - accuracy: 0.4537
Epoch 6/20
5041/5041 [==============================] - 21s 4ms/step - loss: 1.5121 - accuracy: 0.4552
Epoch 7/20
5041/5041 [==============================] - 21s 4ms/step - loss: 1.5130 - accuracy: 0.4560
Epoch 8/20
5041/5041 [==============================] - 22s 4ms/step - loss: 1.5142 - accuracy: 0.4550
Epoch 9/20
5041/5041 [==============================] - 22s 4ms/step - loss: 1.5106 - accuracy: 0.4566
Epoch 10/20
5041/5041 [==============================] - 21s 4ms/step - l

In [34]:
model.evaluate(X_test_from, y_test_from, batch_size = 128)

1261/1261 [==============================] - 4s 3ms/step - loss: 1.3984 - accuracy: 0.4865


[1.398398756980896, 0.4864884912967682]